<a href="https://colab.research.google.com/github/aminaalisheva/Word-Embeddings-on-Azerbaijani-dataset/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.corpus import reuters
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_model = glove2word2vec("../glove/vectors.txt", "w2v.txt")
w2v_model = Word2Vec.load("../word2vec3.model")

/tmp/ipykernel_11439/122502440.py:2: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove_model = glove2word2vec("../glove/vectors.txt", "w2v.txt")


In [ ]:
words_test = ['bank', 'bazar', 'pul', 'ticarət', 'səhm', 'qızıl', 'inkişaf', 'qiymət', 'sərmayə', 'valyuta']
for word in words_test:
    synonyms = glove_model.most_similar(word, topn=5)
    synonyms = [i[0] for i in synonyms]
    print(f"{word}:", synonyms)

AttributeError: 'tuple' object has no attribute 'most_similar'

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/hajili/azsci_topics/train.csv")

In [ ]:
df = df.dropna()
top_topics = df['topic'].value_counts().nlargest(4).index
df = df[df['topic'].isin(top_topics)]
df.head()

,title,topic,subtopic
3,Birbaşa xarici investisiyaların İordaniyanın i...,İqtisad elmləri,Dünya iqtisadiyyatı
4,Uşaqlarda çənə sümüyünün travmatik zədələnmələ...,Tibb elmləri,Stomatologiya
5,Çıxmayan ortodontik qurğularla müalicə zamanı ...,Tibb elmləri,Stomatologiya
6,XIX əsrin ikinci yarısında Cənubi Azarbaycan ə...,Filologiya elmləri,Azərbaycan ədəbiyyatı
7,Bəkir Çobanzadənin elmi və bədii yaradıcılığın...,Filologiya elmləri,Folklorşünaslıq


In [ ]:
df["topic"].value_counts()

topic
Filologiya elmləri    1008
Tibb elmləri           661
İqtisad elmləri        655
Texnika elmləri        550
Name: count, dtype: int64

In [ ]:
docs = list(df["title"])
targets = list(df["topic"])


In [ ]:
# Task 5: Text Classification using Logistic Regression
# Example classification: categories from Reuters
# targets = [reuters.categories(fid)[0] if reuters.categories(fid) else 'none' for fid in reuters.fileids()[:500]]
X_train, X_test, y_train, y_test = train_test_split(docs, targets, test_size=0.3, random_state=42)

In [ ]:
# Vectorization methods
vectorizers = {
    'Count': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

for name, vect in vectorizers.items():
    X_train_vect = vect.fit_transform(X_train)
    X_test_vect = vect.transform(X_test)
    clf = LogisticRegression(max_iter=200)
    clf.fit(X_train_vect, y_train)
    y_pred = clf.predict(X_test_vect)
    print(f'Classification Report using {name} Vectorizer:\n', classification_report(y_test, y_pred))

# Using Word2Vec & GloVe embeddings as features

def doc_embedding(model, documents):
    embeddings = []
    for doc in documents:
        words = [word for word in doc.lower().split() if word in model]
        if words:
            embeddings.append(np.mean(model[words], axis=0))
        else:
            embeddings.append(np.zeros(16))
    return np.array(embeddings)

for name, model in [('Word2Vec', w2v_model.wv), ('GloVe', glove_model)]:
    X_train_emb = doc_embedding(model, X_train)
    X_test_emb = doc_embedding(model, X_test)
    clf = LogisticRegression(max_iter=200)
    clf.fit(X_train_emb, y_train)
    y_pred = clf.predict(X_test_emb)
    print(f'Classification Report using {name} Embeddings:\n', classification_report(y_test, y_pred))


Classification Report using Count Vectorizer:
                     precision    recall  f1-score   support

Filologiya elmləri       0.93      0.99      0.96       316
   Texnika elmləri       0.95      0.91      0.93       162
      Tibb elmləri       0.98      0.92      0.95       196
   İqtisad elmləri       0.95      0.95      0.95       189

          accuracy                           0.95       863
         macro avg       0.96      0.94      0.95       863
      weighted avg       0.95      0.95      0.95       863

Classification Report using TF-IDF Vectorizer:
                     precision    recall  f1-score   support

Filologiya elmləri       0.97      0.98      0.97       316
   Texnika elmləri       0.94      0.94      0.94       162
      Tibb elmləri       0.97      0.95      0.96       196
   İqtisad elmləri       0.95      0.95      0.95       189

          accuracy                           0.96       863
         macro avg       0.96      0.95      0.96       863


/home/jafar/ada/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jafar/ada/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jafar/ada/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
